In [ ]:
import random
import re
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def unknown():
    response = ["Could you please re-phrase that? ", "...", "Sounds about right.", "What does that mean?"][random.randrange(4)]
    return response

def message_probability(user_message, recognised_words, single_response=False, required_words=[]):
    message_certainty = 0
    has_required_words = True

    for word in user_message:
        if word in recognised_words:
            message_certainty += 1

    percentage = float(message_certainty) / float(len(recognised_words))

    for word in required_words:
        if word not in user_message:
            has_required_words = False
            break

    if has_required_words or single_response:
        return int(percentage * 100)
    else:
        return 0

def gpt2_response(user_input):
    # Use GPT-2 for generating a response
    model_name = "gpt2"  # You can experiment with other GPT-2 variants
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)

    input_ids = tokenizer.encode(user_input, return_tensors="pt")

    output = model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text

def check_all_messages(message):
    highest_prob_list = {}

    def response(bot_response, list_of_words, single_response=False, required_words=[]):
        nonlocal highest_prob_list
        highest_prob_list[bot_response] = message_probability(message, list_of_words, single_response, required_words)

    response('Hello!', ['hello', 'hi', 'hey', 'sup', 'hola'], single_response=True)
    response('See you!', ['bye', 'goodbye'], single_response=True)
    response('Thats great!', ['fine', 'good'], single_response=True)
    response('My name is Pikachu', ['what', 'is', 'your', 'name'], required_words=['your','name'])
    response('My name is Pikachu and I am an AI chatbot', ['who', 'are', 'you'], required_words=['who','you'])
    response('I\'m doing fine, and you?', ['how', 'are', 'you'], required_words=['how','are','you'])
    response('You\'re welcome!', ['thank', 'thanks'], single_response=True)
    response('yes! please How may I help you', ['i','need','your','help'],required_words=['need','help'])
    response('yes! please what you want to know', ['i', 'want', 'to', 'know', 'something'], required_words=['want','know'])
    response('yes! please which information you need', ['i', 'need', 'some', 'information'], required_words=['information'])
    response('yes! you can get admission in comsats if you have passed out from higher school for more info please visit https://admissions.comsats.edu.pk/ ', ['can', 'i', 'get', 'admission','in','comsats'], required_words=['get','admission'])
    response('Through this URL: https://www.comsats.edu.pk/ You can see the comsats upcoming Test Dates', ['what', 'are', 'the', 'upcoming','Comsats','test', 'Dates'], required_words=['comsats','test'])
    response('Here is the URL:https://www.eduvision.edu.pk/entry-test/comsats-nts-test-pattern-deadline-weightage-aggregate-merit for the current subjects weightage for test', ['what', 'is', 'current', 'subject', 'weightage', 'for', 'test'],required_words=[ 'subject', 'weightage','test'])
    response('Here is the complete comsats islamabad faculty list: http://ww3.comsats.edu.pk/faculty/FacultyAtoZ.aspx', ['How', 'is','the','faculty', 'of', 'comsats', 'islamabad'],required_words=['faculty', 'comsats'])
    response('Here is the URL for comsats university islamabad prospectus: https://admissions.comsats.edu.pk/Home/Prospectus', ['tell', 'me', 'about', 'comsats', 'university', 'Islamabad', 'prospectus'],required_words=['about','comsats'])
    response('You can get admission in the university by scoring good marks in the nts test and meeting its admission merit',['How','cam','i','get','admission','in','the','university'],required_words=['admission','university'])
    response('Comsats is located on Park Road,Tarlai Kalan for more precise location follow this url https://maps.app.goo.gl/NZcESuaYwSVfzi8fA', ['where','is','comsats','located'],required_words=['where','comsats'])
    response('I\'m doing great. How about you?', ['how', 'was', 'your', 'day'], required_words=['your','day'])

    best_match = max(highest_prob_list, key=highest_prob_list.get)
    return unknown() if highest_prob_list[best_match] < 1 else best_match

def get_response(user_input):
    split_message = re.split(r'\s+|[,;?!.-]\s*', user_input.lower())
    response = check_all_messages(split_message)

    if response == 'unknown':
        response = gpt2_response(user_input)

    return response

# Testing the response system
while True:
    user_input = input('You: ')
    print('Pikachu: ' + get_response(user_input))